In [2]:
import random
from utils import (zufällige_primzahl, teiler_fremde_zahlen, multiplikative_inverse, 
                   verschlüsseln, entschlüsseln, kontrolle)

In [3]:
RANDOM_SEED = 42

# Ich verwende einen Random Seed, damit das Random Modul bei jeder Ausführung dieselben Werte zurückgibt.
# Wenn man aber andere Werte aus dem Random Modul generieren möchte, sollte man entweder einen anderen Seed 
# verwenden oder einfach den Seed komplett löschen, um bei jeder Ausführung neue Zufallswerte zu erhalten.

### RSA Verfahren

1 - Wähle zwei verschiedene Primzahlen $p$ und $q$ sodass:
$$p, q \in \mathbb{P} \, \land \, p \neq q$$

In [4]:
# zufällige p und q generieren

random.seed(RANDOM_SEED)
p: int = zufällige_primzahl(3, 13)
q: int = zufällige_primzahl(3, 13, außer=p)

print(f"{p = } | {q = }")

p = 3 | q = 5


2 - Berechne das Produkt $N$:
$$N = p.q$$

dieses N ist unsere Modulo Basis.

In [5]:
# N berechnen

N: int = p * q

print(f"{N = }")

N = 15


3 - Berechne die Anzahl der zu N teilerfremden Zahlen, die auch kleiner als N sind, mit der folgenden Formel:
$$ \varphi(N) = \varphi(p, q) = (p - 1)(q - 1)$$

wobei, $\varphi(n)$ ist definiert als:

$$ \varphi(n) = |\{k \in \mathbb{N} \, | \, 1 ≤ k < n \, \land \, ggT(k, n) = 1 \}| $$

einfacher gesagt, Anzahl aller zu $n$ teilerfremden Zahlen, die kleiner als $n$ sind.

Beispiel:

In unserem Beispiel haben wir $N = 15$, d.h. wir betrachten alle Zahlen $\in \mathbb{N}$ die kleiner als N und zu N teilerfremd sind:

$$ N = 15 = |\{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15\}|$$


$\{1, 3, 5, 15\}$ sind teiler von 15 $\Rightarrow$ nicht teilerfremd.


$\{6, 9, 12\}$ hat den ggT $3$ mit $15$, i.e. $ggT([6, 9, 12], 15) = 3 \, \Rightarrow$ nicht teilerfremd.

$\{10\}$ hat den ggT $5$ mit $15$, i.e. $ggT(10, 15) = 5 \, \Rightarrow$ auch nicht teilerfremd.

dann haben wir nur noch $\{1, 2, 4, 7, 8, 11, 13, 14\}$ übrig, die mit $15$ ggT = 1 haben bzw. zu $15$ teilerfremd sind, und die Anzahl dieser Zahlen ist $\varphi(15) = 8$


Wie gesagt, das gleiche kann auch einfach mit der Formel $ \varphi(N) = (p - 1)(q - 1)$ berechnet werden.


**Hinweis**: Die Begriffe "Teilerfremd" und "$ggT = 1$" sind synonym und beschreiben dasselbe Konzept.

In [6]:
# phi(N) berechnen

phi: int = (p - 1) * (q - 1)

print(f"phi(N) = {phi}")

phi(N) = 8


4 - Wähle eine Encryption Exponenten $e$, sodass:
$$ 1 < e < \varphi(N) \, \land \, ggT(e, \, \varphi(N)) = 1$$

d.h. irgendeine $e$, der Teilerfremd zu $\varphi(N)$ ist.

In Unserem Fall wären $\{3, 5, 7\}$ alle möglichen $e$ aus ($1 < \{2, 3, 4, 5, 6, 7\} < \varphi(N) = 8)$, die zu $8$ teilerfremd sind, weil $ggT([3, 5, 7], \, 8) = 1$

Wir können hier beliebig ein $e$ auswählen, z.B. $e = 7$

In [7]:
# e berechnen

random.seed(RANDOM_SEED)

e_list: list[int] = teiler_fremde_zahlen(phi)
# Diese Funktion vergleicht die Teiler aller Zahlen zwischen 2 und phi(N) mit den Teilern 
# von phi(N), siehe utils.py für Details.

# wähle eine zufällige Zahl e aus e_list
e: int = random.choice(e_list)

print(f"Alle mögliche e = {e_list}")
print(f"{e = }")

Alle mögliche e = [3, 5, 7]
e = 7


Jetzt haben wir schon unseren öffentlichen Schlüssel:
$$Pubkey = (e, N)$$

In [8]:
# der öffentliche Schlüssel

print(f"Pubkey = ({e}, {N})")

Pubkey = (7, 15)


5 - Berechne die Decryption Exponenten $d$ sodass:
$$e\,.\,d \mod \varphi(N) \equiv 1$$
d.h. wenn wir die beide Zahlen $d$ und $e$ miteinander multiplizieren und daraus modulo $\varphi(N)$ nehmen, sollte 1 als Ergebnis rauskommen.

z.B. 

$$7\, . \, 23 \mod 8 = 161 \mod 8 \equiv 1$$

**Hinweis**: Modulo berechnet man indem man eine Zahl durch die Basis von Module teilt und nur den Rest als Ergebnis mit dem Symbol "$\equiv$" angibt.

z.B. 
$$7 \mod 5 \equiv 2$$
$$25 \mod 5 \equiv 0$$
$$10 \mod 12 \equiv 10$$

Wie berechnen wir aber den multiplikativen Inverse wenn wir nur $e$ und $\varphi(N)$ kennen?

Es gibt verschiedene Algorithmen, die den Inversen in diesem Fall einfach berechnen lassen. Man kann aber auch manuell vorgehen. Zum Beispiel versuchen wir für alle natürlichen Zahlen ab 2,
ob das Produkt dieser Zahl mit $d$ darauf modulo $\varphi(N)$, den Rest 1 ergibt. So könnten theoretisch unendlich viele Werte für den Inversen gefunden werden, aber wir nehmen einfach den ersten Wert, den wir finden, mit der Bedingung, $$d \neq e \, \land \, d \neq N$$

Genause macht **'multiplikative_inverse'** Funktion aus Utils.

Wenn wir manuell so berechnen, bekommen wir als erstes den Wert $23$, sodass:
$$7\, . \, 23 \mod 8 = 161 \mod 8 \equiv 1$$

In [9]:
# d berechnen

d: int = multiplikative_inverse(e, phi, N)

print(f"{d = }")

d = 23


Somit haben wir unseren privaten Schlüssel:
$$ Privkey = (d, \, \varphi(N))$$

In [10]:
# der private Schlüssel

print(f"Privkey = ({d}, {phi})")

Privkey = (23, 8)


### Verschlüsselung

Wir wollen z.B. unsere Nachricht $m = 2$ ($m < N$ sein) verschlüsseln, dafür brauchen wir nur den öffentlichen Schlüssel nämlich $Pubkey = (7, 15)$

$$e : m \mapsto e_{Pubkey}(m) = m^e \mod N = c$$
$$\Rightarrow c = 2^7 \mod 15 = 128 \mod 15 = 8$$

$\Rightarrow$ Unsere verschlüsselte Nachricht $c$ ist: $8$

Genau das gleiche macht die Funktion '**verschlüsseln**' aus Utils

In [11]:
# m verschlüsseln

m = 2
c: int = verschlüsseln(m, e, N)

print(f"Plaintext: {m = }")
print(f"Verschlüsselter Text: {c = }")

Plaintext: m = 2
Verschlüsselter Text: c = 8


### Entschlüsselung

Jetzt wollen wir die verschlüsselte Nachricht $c = 8$ entschlüsseln, dafür brauchen wir unseren privaten Schlüssel, nämlich $Privkey = (23, 8)$

$$d : c \mapsto d_{Privkey}(c) = c^d \mod N = m$$
$$\Rightarrow m = 8^{23} \mod 8 = 2$$

$\Rightarrow$ Unsere entschlüsselte Nachricht $m$ ist $2$, was dem urspüngilichen Plaintext entspricht.

Genau das gleiche macht die Funktion '**entschlüsseln**' aus Utils

In [12]:
# c entschlüsseln

m: int = entschlüsseln(c, d, N)

print(f"Text nach dem Entschlüsseln: {m = }")

Text nach dem Entschlüsseln: m = 2


Jetzt wollen wir kontrollieren, ob das immer gelten würde, falls $m < N$ ist. In der Funktion **kontrolle** aus Utils, prüfen wir für jede einzilne Zahl m aus (1; N), ob der Rückgabewert von 'entschlüsseln' wieder den Plaintext ergibt.

In [13]:
# kontrolle

print(kontrolle(e, d, N))

True


Sobald der Rückgabewert der 'kontrolle' Funktion "TRUE" ist, heißt das also, dass dieses Algorithm wird immer für alle $m < N$ funktionieren.